In [2]:
import numpy as np
import tensorflow as tf
import random
import pandas as pd

In [231]:
tf.reset_default_graph()


In [232]:
filename = "dataset_kor_OSX/교통사망사고정보/Kor_Train_교통사망사고정보(12.1~17.6).csv"
traffic = np.loadtxt(filename
                     , delimiter=',', dtype=np.str_, skiprows=True)

In [233]:
traffic1= traffic

In [234]:
traffic1.shape

(25037, 27)

In [235]:
#발생년, 발생분 제외
index =[]
for i in range(27):
    if(i==0 or i == 2):
        index.append(False)
    else:
        index.append(True)
    
print(index)    

[False, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]


In [236]:
traffic1= traffic1[:, index]

In [237]:
traffic1

array([['2017063023', '야간', '금', ..., '1937456', '127.1341305',
        '37.4357039'],
       ['2017063021', '야간', '금', ..., '1696405', '127.1312203',
        '35.2625338'],
       ['2017063021', '야간', '금', ..., '1865697', '126.3799216',
        '36.7841423'],
       ...,
       ['2012010102', '야간', '일', ..., '1942819', '127.0348466',
        '37.48369509'],
       ['2012010101', '야간', '일', ..., '1737729', '126.9094055',
        '35.63407671'],
       ['2012010101', '야간', '일', ..., '1957198', '126.9318767',
        '37.61284962']], dtype='<U15')

In [238]:
traffic1[:, 0]

array(['2017063023', '2017063021', '2017063021', ..., '2012010102',
       '2012010101', '2012010101'], dtype='<U15')

In [239]:
traffic1.shape

(25037, 25)

In [240]:
type(len(traffic1))

int

In [241]:
#발생년월일시 중 발생 월말 빼오기
month_array= []
num=0
for i in traffic1[:,0]:
    month_array.append(i[4:6])
    num+=1


In [242]:
traffic1[:, 0]= month_array

In [243]:
#make last 4 features float and normalize
location_matrix = traffic1[:,-4:].astype(float)

for i in range(location_matrix.shape[1]):
    column = location_matrix[:,i]
    numerator = column - np.min(column)
    denominator= np.max(column)-np.min(column)
    normalization = numerator / (denominator+ 1e-7)
    location_matrix[:,i]=normalization

In [244]:
location_matrix.shape

(25037, 4)

In [245]:
traffic1[:,-4:] = location_matrix

In [246]:
traffic1

array([['06', '야간', '금', ..., '0.9404418599570', '0.7079475328480',
        '0.9415098802989'],
       ['06', '야간', '금', ..., '0.8234356152813', '0.7077221581647',
        '0.8261818287141'],
       ['06', '야간', '금', ..., '0.9056100147803', '0.6495393162425',
        '0.9069321314998'],
       ...,
       ['01', '야간', '일', ..., '0.9430450621433', '0.7002586875737',
        '0.9440567266943'],
       ['01', '야간', '일', ..., '0.8434943001861', '0.6905441496784',
        '0.8458992543728'],
       ['01', '야간', '일', ..., '0.9500246340687', '0.6922843873194',
        '0.9509108336500']], dtype='<U15')

In [247]:
traffic1.shape

(25037, 25)

In [248]:
y_data = traffic1[:, 1]

In [249]:
y_data

array(['야간', '야간', '야간', ..., '야간', '야간', '야간'], dtype='<U15')

In [250]:
#y 데이터로 쓸 컬럼 제거
exclude_day_night_mask=[]
for i in range(len(traffic1[0])):
    if i != 1:
        exclude_day_night_mask.append(True)
    else:
        exclude_day_night_mask.append(False)

In [251]:
traffic1= traffic1[:,exclude_day_night_mask]

In [252]:
traffic1

array([['06', '금', '1', ..., '0.9404418599570', '0.7079475328480',
        '0.9415098802989'],
       ['06', '금', '1', ..., '0.8234356152813', '0.7077221581647',
        '0.8261818287141'],
       ['06', '금', '1', ..., '0.9056100147803', '0.6495393162425',
        '0.9069321314998'],
       ...,
       ['01', '일', '1', ..., '0.9430450621433', '0.7002586875737',
        '0.9440567266943'],
       ['01', '일', '1', ..., '0.8434943001861', '0.6905441496784',
        '0.8458992543728'],
       ['01', '일', '1', ..., '0.9500246340687', '0.6922843873194',
        '0.9509108336500']], dtype='<U15')

In [253]:
traffic1.shape

(25037, 24)

In [254]:
#category형 데이터만 true, 수치형은 false 
column_mask =[]
for i in range(traffic1.shape[1]):
    if(i==2 or i == 3 or i ==4 or i==5 or i==6 or i==20 or i==21 or i ==22 or i==23
      ):
        column_mask.append(False)
    else:
        column_mask.append(True)

In [255]:
column_mask

[True,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False]

In [256]:
#수치형 데이터들 모아놓기
not_onehot=pd.DataFrame(traffic1[:,[not i for i in column_mask]])

In [257]:
#only 카테고리형 데이터
make_onehot= traffic1[:, column_mask]

In [258]:
make_onehot

array([['06', '금', '경기', ..., '승용차', '보행자', '보행자'],
       ['06', '금', '전남', ..., '승용차', '승용차', '승용차'],
       ['06', '금', '충남', ..., '승용차', '보행자', '보행자'],
       ...,
       ['01', '일', '서울', ..., '중형', '보행자', '보행자'],
       ['01', '일', '전북', ..., '중형', '승용차', '소형'],
       ['01', '일', '서울', ..., '중형', '보행자', '보행자']], dtype='<U15')

In [259]:
pdData = pd.DataFrame(make_onehot)

In [260]:
df = pd.DataFrame()
for i in range(len(pdData.columns)):
    one_hot = pd.get_dummies(pdData.iloc[:, i])
    df=pd.concat([df, one_hot], axis =1)

In [261]:
df.shape

(25037, 399)

In [262]:
#one-hot 된 카테고리형 데이터 + 일반 수치형 데이터
df = pd.concat([df, not_onehot], axis = 1)

In [263]:
df.shape

(25037, 408)

In [264]:
# make pandas df as ndarray
x_data = df.as_matrix()

In [265]:
# one_hot = pd.get_dummies(pdData.iloc[:, i])
y_df = pd.DataFrame(y_data)
y_onehot = pd.get_dummies(y_df)

In [266]:
y_data = y_onehot.as_matrix()

In [267]:
y_data.shape

(25037, 2)

In [268]:
x_data.shape

(25037, 408)

### Randomly Extract Test, Validation, Train Data

In [269]:
train_num = int(len(x_data)*0.6)
validation_num = int((len(x_data)-train_num)*0.5)
test_num = len(x_data)-train_num-validation_num

train_index = random.sample(range(0, len(x_data)), train_num)
validation_index = random.sample(range(0 , len(x_data)- train_num), validation_num)

In [270]:
x_train= x_data[train_index, :]
y_train = y_data[train_index, :]
entire_list = [i for i in range(len(x_data))]
for i in train_index:
    entire_list.remove(i)

In [271]:
x_test_validation = x_data[entire_list, :]
y_test_validation = y_data[entire_list, :]

In [272]:
x_validation= x_test_validation[validation_index, :]
y_validation=y_test_validation[validation_index, :]
validation_list = [i for i in range(len(x_data)-train_num)]
for i in validation_index:
    validation_list.remove(i)    

In [306]:
x_test=x_test_validation[validation_list, :]
y_test= y_test_validation[validation_list, :]

In [307]:
x_train.shape[1]
y_train.shape[1]

2

### 여기부터 TensorFlow

주 야간 예측

In [276]:
# parameters
learning_rate = 0.001

In [285]:
tf.reset_default_graph()
tf.get_default_graph()


In [290]:
x_train.shape[1]
y_train.shape[1]

2

In [315]:
with tf.Graph().as_default():
    X = tf.placeholder(tf.float32, [None, x_train.shape[1]], name="input_x_placeholder")
    Y = tf.placeholder(tf.float32, [None, y_train.shape[1]], name="input_y_placeholder")
    # weights & bias for nn layers
    W1 = tf.get_variable("W1", shape=[x_train.shape[1], 500],
                         initializer=tf.contrib.layers.xavier_initializer())
    b1 = tf.Variable(tf.random_normal([500]))
    L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

    W2 = tf.get_variable("W2", shape=[500, 500],
                         initializer=tf.contrib.layers.xavier_initializer())
    b2 = tf.Variable(tf.random_normal([500]))
    L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

    W3 = tf.get_variable("W3", shape=[500, 500],
                         initializer=tf.contrib.layers.xavier_initializer())
    b3 = tf.Variable(tf.random_normal([500]))
    L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)

    W4 = tf.get_variable("W4", shape=[500, 500],
                         initializer=tf.contrib.layers.xavier_initializer())
    b4 = tf.Variable(tf.random_normal([500]))
    L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)

    W5 = tf.get_variable("W5", shape=[500, y_train.shape[1]],
                         initializer=tf.contrib.layers.xavier_initializer())
    b5 = tf.Variable(tf.random_normal([y_train.shape[1]]))
    hypothesis = tf.matmul(L4, W5) + b5

    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train = optimizer.minimize(cost)
    
    is_correct = tf.equal(tf.arg_max(hypothesis, 1), tf.arg_max(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    for step in range(1000):
        cost_val, _ = sess.run(
            [cost, train],
            feed_dict={X: x_train, Y: y_train}
        )
        if step % 100 == 0:
            print(step, "Cost: ", cost_val)
    print("Accuracy: ", accuracy.eval(session=sess, feed_dict={
          X: x_test, Y: y_test}))

    

0 Cost:  1.9960903
100 Cost:  0.5413009
200 Cost:  0.20260234
300 Cost:  0.18955988
400 Cost:  0.01644826
500 Cost:  0.0034590878
600 Cost:  0.0021510131
700 Cost:  0.001840065
800 Cost:  0.0017227095
900 Cost:  0.0016899531
Accuracy:  0.5954473
